In [302]:
import pandas as pd
import numpy as np
import os
import json

In [303]:
DATA_DIR = "data/"
FILE_NAME = "data.csv"
FINAL_DATA = "rearranged_data.csv"
DATA_SPECS = "data_specs.json"

In [ ]:
with open(DATA_SPECS, 'r') as f:
    DATA_SPECS_DICT = json.load(f)

In [304]:
# Load data
df = pd.read_csv(os.path.join(DATA_DIR, FILE_NAME), delimiter=";")

In [305]:
# function to copy serial
def copy_serial(row):
    if not pd.isnull(row["ZG04"]):
        row["SERIAL"] = row["ZG04"]
    elif not pd.isnull(row["ZG05"]):
        row["SERIAL"] = row["ZG05"]
    return row

In [306]:
# move serial to serial from w01
df = df.apply(lambda row: copy_serial(row), axis=1)

In [307]:
# Drop lines where we have no serial number
df = df[~pd.isnull(df["SERIAL"])]

In [308]:
# Function to extract group
serial_group = dict()
def extract_variable(row):
    if not pd.isnull(row["ZG04"]):
        serial_group.update({row["SERIAL"]:"MS"})
    elif not pd.isnull(row["ZG05"]):
        serial_group.update({row["SERIAL"]:"AL"})
    
def assign_variable(var):
    return serial_group[var]

In [1]:
%%capture
# Extract group
df.apply(lambda row: group_variable(row), axis=1)

NameError: name 'df' is not defined

In [310]:
# Drop some unnecessary columns
df.drop(DATA_SPECS_DICT["drop_vars"], axis=1, inplace=True)

In [311]:
# Find all cases that have completed all the stuff
def collect_complete_cases():
    complete_cases = []
    for indiv in df["SERIAL"].unique():
        if df[df["SERIAL"]== indiv].shape[0] == 12:
            complete_cases.append(indiv)
    return complete_cases

complete_cases = collect_complete_cases()
df = df[df["SERIAL"].isin(complete_cases)]

In [312]:
value_vars_drop = ['SERIAL',"QUESTNNR"]
value_vars = [x for x in df.columns if x not in value_vars_drop]
df = pd.melt(df, id_vars=["SERIAL","QUESTNNR"], value_vars=value_vars)

In [313]:
# Drop variables without anser
df = df[~pd.isnull(df["value"])]

In [314]:
# Add group variable
df["GROUP"] = df["SERIAL"].apply(lambda val: serial_group[val])

In [315]:
# Rearrange table
df = df[["SERIAL", "GROUP", "QUESTNNR", "variable", "value"]]

In [320]:
# Store to new excel
df.to_excel(os.path.join(DATA_DIR, FINAL_DATA), index=False)